In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd "/content/drive/My Drive/EVA4/Notebooks/S11"

/content/drive/My Drive/EVA4/Notebooks/S11


In [3]:
from vision import dataloaders
from vision.utils import Helper
from vision.models.resnet import ResNet18
from vision.models.customeResNet import CustomModel
from vision.imagetransforms import TorchTransforms,  album_transforms
import torch
from torchvision import  transforms
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from vision.train_test_model import RunModel
from vision.lr_finder import LRFinder
from torch.optim.lr_scheduler import ReduceLROnPlateau
from vision import plot_grad
from vision import train_test_model

  pip install torch-lr-finder -v --global-option="amp"


In [0]:
helper = Helper()

In [0]:
Net = CustomModel()

In [6]:
size =  (3, 32, 32)
helper.model_summary(Net, size)

RuntimeError: ignored